# Rozmawiaj z bazą danych!

# Sposób działania:

Możesz zadawać różnorodne pytania dotyczące książek w bazie, bazując na ich tytule, autorze, opisie i kategorii. 
Oto kilka przykładowych typów zapytań, które możesz przetestować:

- "Szukam książki o uczeniu maszynowym."
- "Jakie książki polecasz do nauki Excela?"
- "Czy masz coś na temat cyberbezpieczeństwa?"
- "Czy masz książki napisane przez 'Eric Matthes'?"
- "Czy masz książkę '100 sposobów na Excel'?"
- "Czy jest dostępna książka 'Python dla zaawansowanych'?"
- "Szukam książki o tym, jak tworzyć wykresy w Excelu."
- "O czym jest książka 'Clean Code'?"
- "Jakie zagadnienia porusza książka 'Python Crash Course'?"


# Uwagi:

1. Model OpenAI nie skanuje całego pliku XLSX:
Model nie przetwarza bezpośrednio całej bazy książek – działa na zasadzie komunikacji z funkcją find_books(), która lokalnie przeszukuje plik Excela przy użyciu pandas i zwraca tylko odpowiednie wyniki.

2. Jak działa wyszukiwanie?
- Excel jest wczytywany lokalnie przy użyciu pandas, ale OpenAI go nie widzi.
- Funkcja find_books() filtruje tylko te książki, które pasują do zapytania użytkownika.
- Tylko wybrane książki (np. 5 wyników) są wysyłane do modelu OpenAI.
- Model OpenAI nie ma dostępu do całego Excela, widzi tylko te wyniki, które mu przekażę.

3. Dlaczego to działa efektywnie?
- Nie przekraczam limitu tokenów.
- Nie marnuję zasobów na nieistotne dane.
- Mogę mieć dużą bazę książek (np. 10000+ rekordów) bez problemów z wydajnością.
- Jak wyżej, model działa szybciej, bo przetwarza tylko filtrowane wyniki.

4. Co by się stało, gdyby model miał cały plik XLSX?
- Limit tokenów szybko zostałby przekroczony (nawet 500 rekordów może zająć za dużo miejsca).
- Wyszukiwanie byłoby wolne, bo OpenAI musiałby przeanalizować ogromną ilość danych.
- Koszt generacji odpowiedzi byłby wyższy.

5. Baza książek jest fikcyjna, została stworzona przy użyciu ChatGPT, model o1

In [1]:
# Import potrzebnych bibliotek

from openai import OpenAI
import json
import pandas as pd
import openpyxl
import re

In [2]:
# Inicjalizacja API OpenAI
client = OpenAI()

In [3]:
# Wczytanie pliku z bazą danych
books_df = pd.read_excel("lista_ksiazek_IT.xlsx")  

In [4]:
# Sprawdzenie fragmentu bazy danych
books_df.head(10)

,Tytuł,Autor,Kategoria,Opis
0,Python dla Początkujących: Podstawy,Adam Kowalski,Programowanie w Pythonie,Podstawowe konstrukcje języka i proste projekt...
1,Tworzenie Skryptów w Pythonie: Praktyczne Proj...,Marta Nowak,Programowanie w Pythonie,Poznaj narzędzia i biblioteki do automatyzacji...
2,Zaawansowane Techniki w Pythonie,Piotr Zawadzki,Programowanie w Pythonie,"Eksploruj wzorce projektowe, optymalizację i m..."
3,Python w Analizie Danych,Anna Lewandowska,Programowanie w Pythonie,"Książka omawia biblioteki numpy, pandas i matp..."
4,Python Web Development: Django & Flask,Maria Nowicka,Programowanie w Pythonie,Praktyczne wprowadzenie do tworzenia stron int...
5,Sztuczna Inteligencja w Pythonie,Paweł Majchrzak,Programowanie w Pythonie,Wprowadzenie do bibliotek TensorFlow i PyTorch...
6,Python: Tworzenie API i Mikrousług,Krzysztof Wróblewski,Programowanie w Pythonie,"Dowiedz się, jak projektować REST-owe API i sk..."
7,Optymalizacja Kodów w Pythonie,Ewa Adamska,Programowanie w Pythonie,"Metody profilowania i usprawniania kodu, aby t..."
8,Java dla Początkujących: Start,Tomasz Wiśniewski,Programowanie w Java,"Podstawowe elementy języka, instalacja środowi..."
9,Java: Programowanie Wielowątkowe,Wojciech Wróbel,Programowanie w Java,Jak efektywnie korzystać z wątków i synchroniz...


In [5]:
# Funkcja do wyszukiwania książek na podstawie zapytania użytkownika
def find_books(query):
    # Szuka w tytule, autorze, kategoriach i opisie
    filtered_books = books_df[
        books_df.apply(lambda row: query.lower() in row.to_string().lower(), axis=1)
    ]

    # Zwraca tylko istotne kolumny
    results = filtered_books[['Tytuł', 'Autor', 'Kategoria', 'Opis']].to_dict(orient="records")

    # Ogranicza do 5 wyników
    return results[:5]

# Definicja funkcji OpenAI
functions = [
    {
        "name": "find_books",
        "description": "Wyszukiwanie książek według tytułu, autora, kategorii lub opisu",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Wyszukiwanie książek np. po tytule, autorze lub kategorii."
                }
            },
            "required": ["query"],
        },
    }
]

# Funkcja do interakcji z AI
def chat_with_ai(question):
    messages = [{"role": "user", "content": question}]

    # Wysyłanie zapytania do OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions=functions
    )

    # Pobranie odpowiedzi z modelu
    response_message = response.choices[0].message
    messages.append(response_message)

    # Jeśli model wygenerował wywołanie funkcji, obsłuż je
    if response_message.function_call:
        function_args = json.loads(response_message.function_call.arguments)
        books = find_books(function_args.get("query"))

        # Dodanie wyniku funkcji do rozmowy
        messages.append(
            {"role": "function",
             "name": "find_books",
             "content": json.dumps(books, ensure_ascii=False)}
        )

        # Kolejne zapytanie do modelu z wynikami funkcji
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
        )

        return response.choices[0].message.content
    else:
        return "❌ Model nie zwrócił wywołania funkcji."

# Wywołanie funkcji
if __name__ == "__main__":
    user_question = input("Wpisz swoje pytanie: ")
    response = chat_with_ai(user_question)
    print("\n🔹 Odpowiedź AI:")
    print(response)


🔹 Odpowiedź AI:
Oto kilka ciekawych książek do nauki Pythona, które mogą Cię zainteresować:

1. "Python dla Początkujących: Podstawy" autorstwa Adama Kowalskiego - książka prezentująca podstawowe konstrukcje języka i proste projekty do nauki Pythona dla początkujących.

2. "Tworzenie Skryptów w Pythonie: Praktyczne Projekty" autorstwa Marty Nowak - podręcznik, który przedstawia narzędzia i biblioteki do automatyzacji zadań za pomocą prostych skryptów w Pythonie.

3. "Zaawansowane Techniki w Pythonie" autorstwa Piotra Zawadzkiego - książka, w której można znaleźć informacje o wzorcach projektowych, optymalizacji i metaprogramowaniu w celu tworzenia wydajniejszych aplikacji.

4. "Python w Analizie Danych" autorstwa Anny Lewandowskiej - poradnik omawiający biblioteki numpy, pandas i matplotlib, prezentujący techniki przetwarzania i wizualizacji danych w Pythonie.

5. "Python Web Development: Django & Flask" autorstwa Marii Nowickiej - praktyczne wprowadzenie do tworzenia stron internetow

# Przykłady użycia:

In [6]:
chat_with_ai("O czym jest książka Unix: Architektura i Programowanie?")

'Książka "Unix: Architektura i Programowanie" autorstwa Roberta Jaworskiego skupia się na procesach, sygnałach, potokach oraz pisaniu aplikacji systemowych w środowisku UNIX. Jest to pozycja z kategorii Systemy Operacyjne, która obejmuje tematy związane z architekturą i programowaniem w systemie UNIX.'

In [ ]:
# Funkcja input do wprowadzenia pytania. W tym wypadku zapytanie dotyczyło polecenia książki o Flutterze
chat_with_ai(input("Zadaj pytanie: "))

'Polecam książkę "Flutter i Dart: Aplikacje Wieloplatformowe" autorstwa Marty Biernackiej. Książka ta skupia się na wydajnych interfejsach, hot reload oraz tworzeniu animacji w frameworku Flutter od Google.'